## 4. Modell Training und Auswahl

In [10]:
import pandas as pd
import pickle

from sklearn.preprocessing import LabelEncoder
import pandas as pd

original_df = pd.read_csv('../data (clean)/cleaned_data.csv')
X = pd.read_csv('../model/feature_matrix.csv')
y = pd.read_csv('../model/target_variable.csv')

# Load the encoders from the file in the 'model' folder
with open('../model/encoders.pkl', 'rb') as file:
    encoders = pickle.load(file)

In [11]:
# Importing necessary libraries
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing the XGBoost Regressor
xgb_model = XGBRegressor()

# Training the model
xgb_model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = xgb_model.predict(X_test)

# Calculating the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
mse

4757645.140852629

In [12]:
# Importing the necessary library to calculate R^2 score
from sklearn.metrics import r2_score

# Calculating the R^2 score
r2 = r2_score(y_test, y_pred)

# Converting R^2 score to percentage
r2_percentage = r2 * 100
r2_percentage

96.36840419405755

In [13]:
import joblib

# Annahme: Das trainierte Modell ist in der Variable 'model' gespeichert
joblib.dump(xgb_model, '../model/trained_model(xgb).joblib')

['../model/trained_model(xgb).joblib']

In [14]:
label_columns = ['Marke', 'Modell', 'Getriebe', 'Kraftstoffart']
current_year = 2023

In [15]:
def predict_car_price(Marke, Modell, Baujahr, Getriebe, Kilometerstand,
                      Kraftstoffart, Verbrauch, Motorgröße):
    # Adding a space before the 'Modell' value to match the original data
    Modell = ' ' + Modell

    # Creating a DataFrame from the input values
    input_data = pd.DataFrame({
        'Marke': [Marke],
        'Modell': [Modell],
        'Baujahr': [Baujahr],
        'Getriebe': [Getriebe],
        'Kilometerstand': [Kilometerstand],
        'Kraftstoffart': [Kraftstoffart],
        'Verbrauch (l/100km)': [Verbrauch],
        'Motorgröße (l)': [Motorgröße]
    })

    # Applying Label Encoding to the categorical columns
    for column in label_columns:
        encoder = encoders[column]
        input_data[column] = encoder.transform(input_data[column])

    # Calculating the age of the car
    input_data['Alter'] = current_year - input_data['Baujahr']

    # Making the prediction using the trained XGBoost model
    predicted_price = xgb_model.predict(input_data)

    return predicted_price[0]

In [ ]:
# Example prediction
example_prediction = predict_car_price(
    Marke='Audi', Modell='A6', Baujahr=2017, Getriebe='Manuell',
    Kilometerstand=25322, Kraftstoffart='Benzin', Verbrauch=4.25,
    Motorgröße=1.4
)